## compute model derived fields (vorticity, steric height)

This notebook can be used to compute steric height and vorticity for regional llc4320 output.

Output is stored in new netCDF files in a "derived" directory.

Requires that an Argo climatology has been downloaded and stored locally.

NOTE: Suggested changes:
- computing strain, MLD, or other parameters
- making code work with other simulation data
- turned this notebook into a script that is called by the parent notebook
- loading the reference T/S data from ERDDAP (or another gridded Argo database)

Initial commit Aug 9, 2021 by kdrushka

In [1]:
%matplotlib inline
import os
# import sys
# import fsspec
import numpy as np
import glob
# import re
import gsw as sw
import xarray as xr
import dask.array as dsa
import xgcm.grid
import netCDF4 as nc4

In [2]:
# --------------------------------------------------------------------
# USER INPUTS:
# specify region from this list:
# WesternMed  ROAM_MIZ  NewCaledonia  NWPacific  BassStrait  RockallTrough  ACC_SMST
# MarmaraSea  LabradorSea  CapeBasin
RegionName = 'ACC_SMST' 


# directory where model data is stored:
llc4320dir = '/data1/adac/mitgcm/netcdf/'
regiondir = llc4320dir + RegionName + '/'
# directory to save derived data to - create if doesn't exist
derivedir = regiondir + 'derived/'
if not(os.path.isdir(derivedir)):
    os.mkdir(derivedir)

In [3]:
# load a single file to get coordinates
fg = sorted(glob.glob(regiondir + '**nc')) # all files
i=0
thisf=fg[i]
print(thisf)
ds = xr.open_dataset(thisf)
    
# mean lat/lon of domain
xav = ds.XC.isel(j=0).mean(dim='i')
yav = ds.YC.isel(i=0).mean(dim='j')
print('center of domain: ', yav.values, ',' , xav.values)

/data1/adac/mitgcm/netcdf/ACC_SMST/LLC4320_pre-SWOT_ACC_SMST_20111221.nc
center of domain:  -55.023106 , 152.29167


In [4]:
# for vorticity calculation, build the xgcm grid:
# see https://xgcm.readthedocs.io/en/latest/xgcm-examples/02_mitgcm.html
grid = xgcm.Grid(ds, coords={'X':{'center': 'i', 'left': 'i_g'}, 
                 'Y':{'center': 'j', 'left': 'j_g'},
                 'T':{'center': 'time'},
                 'Z':{'center': 'k'}})

In [5]:
# load reference file of argo data
# NOTE: could update to pull from ERDDAP or similar
argoclimfile = '/data1/argo/argo_CLIM_3x3.nc'
argods = xr.open_dataset(argoclimfile,decode_times=False) 
# reference profiles: annual average Argo T/S using nearest neighbor
Tref = argods["TEMP"].sel(LATITUDE=yav,LONGITUDE=xav, method='nearest').mean(dim='TIME')
Sref = argods["SALT"].sel(LATITUDE=yav,LONGITUDE=xav, method='nearest').mean(dim='TIME')
# SA and CT from gsw:
# see example from https://discourse.pangeo.io/t/wrapped-for-dask-teos-10-gibbs-seawater-gsw-oceanographic-toolbox/466
Pref = xr.apply_ufunc(sw.p_from_z, -argods.LEVEL, yav)
Pref.compute()
SAref = xr.apply_ufunc(sw.SA_from_SP, Sref, Pref, xav, yav,
                       dask='parallelized', output_dtypes=[Sref.dtype])
SAref.compute()
CTref = xr.apply_ufunc(sw.CT_from_pt, Sref, Tref, # Theta is potential temperature
                       dask='parallelized', output_dtypes=[Sref.dtype])
CTref.compute()
Dref = xr.apply_ufunc(sw.density.rho, SAref, CTref, Pref,
                    dask='parallelized', output_dtypes=[Sref.dtype])
Dref.compute()
print()

In [24]:
# loop through files, then compute steric height, vorticity, etc. on the i/j grid
# fis = range(1)
fis = range(len(fg))
yn = 'y'
for fi in fis:
    # --- select data ---
    thisf=fg[fi]
    
    # check if output file already exists
    fnout = thisf.replace(RegionName + '_' , RegionName + '_derived-fields_')
    fnout = fnout.replace(RegionName + '/' , RegionName + '/derived/')
    
    if (os.path.isfile(fnout) & (yn.lower() == 'y')):
        yn = input(f'\n{fnout} already exists. Overwrite? (this decision will apply to all files) [y/N]')
    if (yn.lower() == 'n'):
        # do nothing
        1
    elif (yn.lower() == 'y'):   
        print(thisf , '(' , fi+1, 'of', len(fis), ')')  
        ds = xr.open_dataset(thisf)

        # create datasets for variables of interest:
        ss = ds.Salt
        tt = ds.Theta
        pp = xr.DataArray(sw.p_from_z(ds.Z,ds.YC))

        # --- compute steric height in steps ---
        # 1. compute absolute salinity and conservative temperature
        sa = xr.apply_ufunc(sw.SA_from_SP, ss, pp, xav, yav, dask='parallelized', output_dtypes=[ss.dtype])
        sa.compute()
        ct = xr.apply_ufunc(sw.CT_from_pt, sa, tt, dask='parallelized', output_dtypes=[ss.dtype])
        ct.compute()
        dd = xr.apply_ufunc(sw.density.rho, sa, ct, pp, dask='parallelized', output_dtypes=[ss.dtype])
        dd.compute()
        # 2. compute specific volume anomaly: gsw.density.specvol_anom_standard(SA, CT, p)
        sva = xr.apply_ufunc(sw.density.specvol_anom_standard, sa, ct, pp, dask='parallelized', output_dtypes=[ss.dtype])
        sva.compute()
        # 3. compute steric height = integral(0:z1) of Dref(z)*sva(z)*dz(z)
        # - first, interpolate Dref to the model pressure levels
        Drefi = Dref.interp(LEVEL=-ds.Z)
        dz = -ds.Z_bnds.diff(dim='nb').drop_vars('nb').squeeze() # distance between interfaces

        # steric height computation (summation/integral)
        # - increase the size of Drefi and dz to match the size of sva
        Db = Drefi.broadcast_like(sva)
        dzb = dz.broadcast_like(sva)
        dum = Db * sva * dzb
        sh = dum.cumsum(dim='k')

        # --- compute vorticity using xgcm and interpolate to X, Y
        # see https://xgcm.readthedocs.io/en/latest/xgcm-examples/02_mitgcm.html
        vorticity = (grid.diff(ds.V*ds.DXG, 'X') - grid.diff(ds.U*ds.DYG, 'Y'))/ds.RAZ
        vorticity = grid.interp(grid.interp(vorticity, 'X', boundary='extend'), 'Y', boundary='extend')

        # --- save derived fields in a new file
        # - convert sh and zeta to datasets
        dout = vorticity.to_dataset(name='vorticity')
        sh_ds = sh.to_dataset(name='steric_height')
        dout = dout.merge(sh_ds)
        # add/rename the Argo reference profile variables
        tref = Tref.to_dataset(name='Tref')
        tref = tref.merge(Sref).rename({'SALT': 'Sref'}).\
            rename({'LEVEL':'zref','LATITUDE':'yav','LONGITUDE':'xav'}).\
            drop_vars({'i','j'})
        # - add ref profiles to dout and drop uneeded vars/coords
        dout = dout.merge(tref).drop_vars({'LONGITUDE','LATITUDE','LEVEL','i','j'})

        # - save netcdf file with derived fields
        netcdf_fill_value = nc4.default_fillvals['f4']
        dv_encoding = {}
        for dv in dout.data_vars:
            dv_encoding[dv]={'zlib':True,  # turns compression on\
                        'complevel':9,     # 1 = fastest, lowest compression; 9=slowest, highest compression \
                        'shuffle':True,    # shuffle filter can significantly improve compression ratios, and is on by default \
                        'dtype':'float32',\
                        '_FillValue':netcdf_fill_value}
        # save to a new file
        print(' ... saving to ', fnout)
        dout.to_netcdf(fnout,format='netcdf4',encoding=dv_encoding)



/data1/adac/mitgcm/netcdf/ACC_SMST/derived/LLC4320_pre-SWOT_ACC_SMST_derived-fields_20111221.nc already exists. Overwrite? (this decision will apply to all files) [y/N] N
